In [111]:
# library dependencies
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import lzma
import pickle
from sklearn.model_selection import train_test_split
import keras
from keras import layers
import tensorflow as tf

In [2]:
# method to store data as serialized binary structure lzma compressed
def can_pickles(data, filename):
    with lzma.LZMAFile(filename, 'wb') as f:
        pickle.dump(data, f, pickle.DEFAULT_PROTOCOL)

# method to retrieve data from a compressed pickle file (created with the method above)
def uncan_pickles(filename):
    with lzma.LZMAFile(filename, 'rb') as f:
        return pickle.load(f)

baseline data

X holds a list of sequences one hot encoded
y holds a list of PTR values as floats

In [3]:
# read the prepared data back
X = uncan_pickles('../data/onehot_x_lung.pickle.xz')
y = uncan_pickles('../data/onehot_y_lung.pickle.xz')

In [112]:
tensor = tf.ragged.constant(X, dtype=tf.int8, ragged_rank=1, row_splits_dtype=tf.int32)

In [39]:
# convert type of target values from string to float
y = np.array(y).astype(float)

In [7]:
X[0]

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       ...,
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0]])

In [33]:
y[0]

7.544

In [114]:
# count of input sequences
len(X)

8201

split data in train and test subsets

verify that the split worked properly

In [35]:
# get a sample
X_train[1]

array([[1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       ...,
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0]])

In [36]:
# get the matching target
y_train[1]

3.746

In [37]:
# find the target value in the raw dataset
for i in range(len(y)):
    if y[i] == 3.746:
        print(i)

6663


In [40]:
# compare if the raw dataset entry matches the subset entry
if X[6663].all() == X_train[1].all():
    print('OK')

OK


model building

In [100]:
def baseline_model():
    # input
    inputs = layers.Input(shape=(None, 4), name='SequenceInput')
    # 1D convolution
    conv = layers.Conv1D(
        filters=128, 
        kernel_size=10, 
        strides=1, 
        activation='relu',
        name='Conv1D'
    )(inputs)
    # batch normalization
    norm = layers.BatchNormalization(name='Normalization1')(conv)
    # maxpool
    pool = layers.MaxPooling1D(
        pool_size=4,
        strides=4,
        name='Pooling'
    )(norm)
    # dropout
    drop = layers.Dropout(rate=0.2, name='Dropout1')(pool)
    # bi-directional LSTM
    bilstm = layers.Bidirectional(
        layers.LSTM(
            units=128, 
            dropout=0
        ),
        merge_mode='sum',
        input_shape=(None, 4),
        name='BiDirectionalLSTM'
    )(drop)
    drop = layers.Dropout(rate=0.2, name='Dropout2')(bilstm)
    # flatten
    flat = layers.Flatten()(drop)
    # fully connected
    dense = layers.Dense(
        units=64,
        activation='relu',
        name='FullyConnected'
    )(flat)
    # batch normalization
    norm = layers.BatchNormalization(name='Normalization2')(dense)
    # dropout
    drop = layers.Dropout(rate=0.2, name='Dropout3')(norm)
    # dense
    outputs = layers.Dense(units=1, name='Output')(drop)

    # model
    model = keras.Model(inputs=inputs, outputs=outputs, name='BaselineModel')
    
    return model

In [101]:
model = baseline_model()

In [95]:
model.summary()

Model: "BaselineModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 SequenceInput (InputLayer)  [(None, None, 4)]         0         
                                                                 
 Conv1D (Conv1D)             (None, None, 128)         5248      
                                                                 
 Normalization1 (BatchNorma  (None, None, 128)         512       
 lization)                                                       
                                                                 
 Pooling (MaxPooling1D)      (None, None, 128)         0         
                                                                 
 Dropout1 (Dropout)          (None, None, 128)         0         
                                                                 
 BiDirectionalLSTM (Bidirec  (None, 256)               263168    
 tional)                                             

In [106]:
keras.utils.plot_model(model, 'baseline.png')
# img = plt.imread('baseline.png')
# plt.figure(figsize=(10,10))
# plt.imshow(img)
# plt.axis('off')
# plt.show()

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [109]:
model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.RMSprop(),
    metrics=keras.metrics.MeanSquaredError(),
)

In [ ]:
X_train_ragged = tf.ragged.constant(X_train, dtype=tf.int8, ragged_rank=1, row_splits_dtype=tf.int32)

In [110]:
model.fit(X_train_ragged, y_train, batch_size=64, epochs=2, validation_split=0.2)

ValueError: Data cardinality is ambiguous:
  x sizes: 2607, 2211, 2607, 2607, 2607, 2607, 2607, 2607, 2607, 2607, 2607, 2607, 2607, 2607, 2607, 2607, 2607, 2607, 2607, 1332, 2607, 2607, 2607, 2607, 2607, 2468, 1481, 2607, 1466, 2198, 2607, 2607, 2607, 1464, 2607, 2037, 1336, 1795, 1738, 1112, 2573, 2607, 2607, 1668, 2607, 2607, 2278, 2607, 1914, 2607, 2607, 2105, 2607, 2607, 1376, 2179, 2607, 2574, 654, 1578, 1830, 2607, 2226, 2607, 2347, 1789, 728, 2607, 2607, 2607, 2381, 2607, 2484, 983, 2607, 926, 2607, 2607, 2358, 2607, 2607, 1913, 2607, 2607, 2068, 1758, 2607, 2607, 1041, 1468, 2607, 2607, 2607, 1108, 2243, 2607, 2607, 939, 1462, 1360, 1377, 2607, 2607, 2607, 2607, 2607, 2603, 2607, 2607, 2607, 675, 1444, 1001, 1049, 2607, 2447, 2607, 669, 2607, 879, 904, 2607, 2607, 2607, 2081, 2607, 1400, 2607, 2607, 2340, 2607, 2607, 2323, 2389, 2282, 2607, 2278, 1847, 2607, 2607, 2607, 2597, 2607, 2607, 1037, 1697, 1778, 2607, 2607, 2338, 824, 2607, 1469, 2607, 2607, 2607, 2061, 2501, 2607, 15

In [50]:
def full_model():
    # input
    inputs = keras.Input(shape=(None, 10))
    # 1D convolution
    conv = keras.Conv1D(
        filters=128, 
        kernel_size=10, 
        strides=1, 
        activation='relu'
    )(inputs)
    # batch normalization
    norm = keras.BatchNormalization()(conv)
    # maxpool
    pool = keras.MaxPooling1D(
        pool_size=4,
        strides=4
    )(norm)
    # dropout
    drop = keras.Dropout(rate=0.2)(pool)
    # bi-directional LSTM
    bilstm = keras.Bidirectional(
        keras.LSTM(
            units=128,
            dropout=0
        ),
        merge_mode='concat'
    )(drop)
    # batch normalization
    norm = keras.BatchNormalization()(bilstm)
    # maxpool
    pool = keras.MaxPooling1D(
        pool_size=4,
        strides=4
    )(norm)
    # dropout
    drop = keras.Dropout(rate=0.2)(pool)
    # flatten
    flat = keras.Flatten()(drop)

    # second input
    inputs2 = keras.Input(shape=(29,))

    # concatenation
    conc = keras.Concatenate(axis=1)([inputs, inputs2])

    # fully connected
    dense = keras.Dense(
        units=64,
        activation='relu'
    )(conc)
    # batch normalization
    norm = keras.BatchNormalization()(dense)
    # dropout
    drop = keras.Dropout(rate=0.2)(norm)
    # dense
    outputs = keras.Dense(units=1)(drop)

    # model
    model = keras.Model(inputs=[inputs, inputs2], outputs=outputs, name='full_model')
    
    return model